In [1]:
import os
import sys
import numpy as np
import pandas as pd
from tqdm import tqdm
from multiprocessing import Pool, cpu_count

In [2]:
sys.path.append("../src/")

In [3]:
from utilities import *

In [4]:
pd.set_option("display.max_columns", None)

### Initial Test

In [5]:
filepath = os.path.join(dir_dict["clean"], "completed_merged.parquet")
df = pd.read_parquet(filepath)

In [6]:
df = df.select_dtypes("number")
df = df.fillna(0)

In [7]:
df = df.drop([col for col in df.columns if "fight_" in col], axis=1)

In [8]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score

In [9]:
X, y = df.drop("winner", axis=1), df["winner"]

In [10]:
np.mean(cross_val_score(LogisticRegression(random_state=42), X, y, cv=5, n_jobs=cpu_count()))

0.5073615160349855